In [2]:
files = '''regsvr32: Successfully registered DLL 'c:\windows\syswow64\quartz.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\qasf.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\qcap.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\qedit.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\qdvd.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\evr.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\msmpeg2vdec.dll'
regsvr32: Successfully registered DLL 'c:\windows\syswow64\mp3dmod.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\quartz.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\qasf.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\qcap.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\qedit.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\qdvd.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\evr.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\msmpeg2vdec.dll'
regsvr32: Successfully registered DLL 'c:\windows\system32\mp3dmod.dll'''

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/s1/1jpfx0m52rj4k7cgqkh7g3q40000gn/T/ipykernel_3435/1304893467.py:1: SyntaxWarning: invalid escape sequence '\w'
  files = '''regsvr32: Successfully registered DLL 'c:\windows\syswow64\quartz.dll'


In [7]:
fnames = []
for line in files.split("\n"):
    rst = line.replace("regsvr32: Successfully registered DLL ", "").replace("'", "")
    fname = rst.split("\\")[-1]
    fnames.append(fname)
fnames = set(fnames)

In [8]:
fnames

{'evr.dll',
 'mp3dmod.dll',
 'msmpeg2vdec.dll',
 'qasf.dll',
 'qcap.dll',
 'qdvd.dll',
 'qedit.dll',
 'quartz.dll'}

In [14]:
!mkdir tmp_dlls

mkdir: tmp_dlls: File exists


In [11]:
import shutil, os

In [17]:
for fname in fnames:
    f_64 = os.path.join(
        "/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/syswow64", 
        fname
    )
    f_32 = os.path.join(
        "/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32", 
        fname
    )
    shutil.copy(
        f_64, f"tmp_dlls/{fname}-64"
    )
    shutil.copy(
        f_32, f"tmp_dlls/{fname}-32"
    )

/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/qcap.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/mp3dmod.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/qdvd.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/quartz.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/msmpeg2vdec.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/qedit.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/evr.dll
/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/qasf.dll


In [25]:
for fname in fnames:
    p1, p2 = fname.split(".")
    
    path_32 = f"/Users/minkexiu/Desktop/new_dlls/{p1}/{p1}.dll"
    path_64 = f"/Users/minkexiu/Desktop/new_dlls/{p1} (1)/{p1}.dll"

    to_32 = f"/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/system32/{p1}.dll" ## 
    to_64 = f"/Users/minkexiu/Library/Application Support/CrossOver/Bottles/steam/drive_c/windows/syswow64/{p1}.dll"

    shutil.copy(
        path_32, to_32
    )
    shutil.copy(
        path_64, to_64
    )
    
    print(fname)

qcap.dll
mp3dmod.dll
qdvd.dll
quartz.dll
msmpeg2vdec.dll
qedit.dll
evr.dll
qasf.dll


In [23]:
!ls "/Users/minkexiu/Desktop/new_dlls/qcap (1)/qcap.dll"

/Users/minkexiu/Desktop/new_dlls/qcap (1)/qcap.dll


In [1]:
from logging import root
import os.path as osp

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from sklearn.metrics import roc_auc_score
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, train_test_split_edges


class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, 128)
        self.conv2 = GCNConv(128, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        return self.conv2(x, edge_index)

    def decode(self, z, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


def get_link_labels(pos_edge_index, neg_edge_index):
    num_links = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(num_links, dtype=torch.float)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train(data, model, optimizer):
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index,
        num_nodes=data.num_nodes,
        num_neg_samples=data.train_pos_edge_index.size(1))
    
    train_neg_edge_set = set(map(tuple, neg_edge_index.T.tolist()))
    val_pos_edge_set = set(map(tuple, data.val_pos_edge_index.T.tolist()))
    test_pos_edge_set = set(map(tuple, data.test_pos_edge_index.T.tolist()))
    if (len(train_neg_edge_set & val_pos_edge_set) > 0) or (len(train_neg_edge_set & test_pos_edge_set) > 0):
        # 训练集负样本与验证集负样本存在交集，或训练集负样本与测试集负样本存在交集
        print('wrong!')

    optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index)
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index).to(data.x.device)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


@torch.no_grad()
def test(data, model):
    model.eval()

    z = model.encode(data.x, data.train_pos_edge_index)

    results = []
    for prefix in ['val', 'test']:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']
        link_logits = model.decode(z, pos_edge_index, neg_edge_index)
        link_probs = link_logits.sigmoid()
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        results.append(roc_auc_score(link_labels.cpu(), link_probs.cpu()))
    return results

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = Planetoid(root='data', name='Cora', transform=T.NormalizeFeatures())
    data = dataset[0]
    ground_truth_edge_index = data.edge_index.to(device)
    data.train_mask = data.val_mask = data.test_mask = data.y = None
    data = train_test_split_edges(data)
    data = data.to(device)

    model = Net(dataset.num_features, 64).to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

    best_val_auc = test_auc = 0
    for epoch in range(1, 101):
        loss = train(data, model, optimizer)
        val_auc, tmp_test_auc = test(data, model)
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            test_auc = tmp_test_auc
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
              f'Test: {test_auc:.4f}')

    z = model.encode(data.x, data.train_pos_edge_index)
    final_edge_index = model.decode_all(z)


if __name__ == "__main__":
    main()

/Users/minkexiu/opt/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Processing...
Done!
/Users/minkexiu/opt/miniforge3/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


wrong!
Epoch: 001, Loss: 0.6930, Val: 0.7086, Test: 0.7060
wrong!
Epoch: 002, Loss: 0.6826, Val: 0.7026, Test: 0.7060
wrong!
Epoch: 003, Loss: 0.7068, Val: 0.7049, Test: 0.7060
wrong!
Epoch: 004, Loss: 0.6779, Val: 0.7083, Test: 0.7060
wrong!
Epoch: 005, Loss: 0.6850, Val: 0.7232, Test: 0.7234
wrong!
Epoch: 006, Loss: 0.6881, Val: 0.7433, Test: 0.7319
wrong!
Epoch: 007, Loss: 0.6888, Val: 0.7366, Test: 0.7319
wrong!
Epoch: 008, Loss: 0.6882, Val: 0.7257, Test: 0.7319
wrong!
Epoch: 009, Loss: 0.6857, Val: 0.7185, Test: 0.7319
wrong!
Epoch: 010, Loss: 0.6811, Val: 0.7128, Test: 0.7319
Epoch: 011, Loss: 0.6749, Val: 0.7088, Test: 0.7319
wrong!
Epoch: 012, Loss: 0.6712, Val: 0.7074, Test: 0.7319
Epoch: 013, Loss: 0.6720, Val: 0.7093, Test: 0.7319
Epoch: 014, Loss: 0.6647, Val: 0.7221, Test: 0.7319
wrong!
Epoch: 015, Loss: 0.6566, Val: 0.7489, Test: 0.7417
Epoch: 016, Loss: 0.6496, Val: 0.7633, Test: 0.7611
wrong!
Epoch: 017, Loss: 0.6420, Val: 0.7649, Test: 0.7630
wrong!
Epoch: 018, Loss: 

In [17]:
# !pip install torch_geometric

In [4]:
import networkx as nx

G = nx.karate_club_graph()

In [6]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils.convert import from_networkx

# 作者：实力
# 链接：https://juejin.cn/post/7225053568152518714
# 来源：稀土掘金
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

In [7]:
data = from_networkx(G)

In [10]:
data

Data(edge_index=[2, 156], club=[34], weight=[156], name='Zachary's Karate Club', num_nodes=34)

In [16]:
data.edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 30, 32, 33,  2, 33,  0,  4

In [14]:
data.club

['Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Officer',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer']

In [18]:
data.x

In [12]:
data.weight.shape

torch.Size([156])

In [19]:
from torch_geometric.utils import degree, to_undirected
from torch_geometric.transforms import ClusterData

# Convert the graph to an undirected graph, so we can aggregate neighbors in both directions.
G = to_undirected(G)

# Compute the degree of each node.
deg = degree(data.edge_index[0], num_nodes=data.num_nodes)

# Use METIS algorithm to partition the graph into multiple subgraphs.
cluster_data = ClusterData(data, num_parts=2, recursive=False, transform=NormalizeFeatures(),
                           degree=deg)

# 作者：实力
# 链接：https://juejin.cn/post/7225053568152518714
# 来源：稀土掘金
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

ImportError: cannot import name 'ClusterData' from 'torch_geometric.transforms' (/Users/minkexiu/opt/miniforge3/lib/python3.9/site-packages/torch_geometric/transforms/__init__.py)

In [21]:
!pip install stellargraph[demos]

zsh:1: no matches found: stellargraph[demos]


In [22]:
!conda install -c stellargraph stellargraph

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::blinker==1.4=py_1
  - conda-forge/noarch::requests==2.27.1=pyhd8ed1ab_0
  - conda-forge/noarch::prompt_toolkit==3.0.27=hd8ed1ab_0
  - conda-forge/noarch::google-auth-oauthlib==0.4.6=pyhd8ed1ab_0
  - conda-forge/noarch::pyasn1-modules==0.2.7=py_0
  - conda-forge/noarch::nbformat==5.1.3=pyhd8ed1ab_0
  - conda-forge/noarch::six==1.16.0=pyh6c4a22f_0
  - conda-forge/noarch::flit-core==3.7.1=pyhd8ed1ab_0
  - conda-forge/noarch::charset-normalizer==2.0.12=pyhd8ed1ab_0
  - conda-forge/noarch::pyparsing==3.0.7=pyhd8ed1ab_0
  - conda-forge/noarch::pure_eval==0.2.2=pyhd8ed1ab_0
  - conda-forge/noarch::gast==0.4.0=pyh9f0ad1d_0
  - conda-forge/noarch::typing_extensions==4.2.0=pyha770c72_1
  - conda-forge/osx-arm64::tensorflow-base==2.7.0=cpu_py39hbdae8f9_0
  - conda-forge/noarch::tensorboard-plugin-wit==1.8.1=pyhd8ed1ab_0
  -


## Package Plan ##

  environment location: /Users/minkexiu/opt/miniforge3

  added / updated specs:
    - stellargraph


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boltons-23.0.0             |   py39hca03da5_0         429 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    bottleneck-1.3.5           |   py39heec5a64_0         100 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    brotli-1.0.9               |       h1a28f6b_7          18 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    brotli-bin-1.0.9           |       h1a28f6b_7          17 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    ca-certificates-2023.08.22 |       hca03da5_0         124 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    certifi-2023.7.22          |   py39hca03da5_0         154 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
 

In [3]:
import torch
from torch_geometric.data import Data


x = torch.tensor([
    [2,1], 
    [5,6], 
    [3,7], 
    [12,0]
], dtype=torch.float)

y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

edge_index = torch.tensor([[0, 2, 1, 0, 3],
                           [3, 1, 0, 1, 2]], dtype=torch.long)


data = Data(x=x, y=y, edge_index=edge_index)

In [2]:
import numpy as np

In [ ]:
2/0.1*(np.log(2**877-2)-np.log(0.01))